## 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Can we forget about the things I said when I was drunk...', "I didn't mean to call you that", "I can't remember what was said"]


## 데이터 정제
- preprocess_sentence() 함수
- 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가면 잘라내기를 권합니다.

In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [71]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> can we forget about the things i said when i was drunk . . . <end>',
 '<start> i didn t mean to call you that <end>',
 '<start> i can t remember what was said <end>',
 '<start> or what you threw at me please tell me <end>',
 '<start> please tell me why <end>',
 '<start> my car is in the front yard <end>',
 '<start> and i am sleeping with my cloths on <end>',
 '<start> i came in throught the window . . . last night <end>',
 '<start> and your . . . gone <end>',
 '<start> gone it s no suprise to me i am my own worst enemy <end>']

## 평가 데이터셋 분리
- tokenize() 함수로 데이터를 Tensor로 변환한 후, 
- sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다.
- 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!

In [77]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, truncating='post', padding='post')  
    print(tensor,tokenizer)
    return tensor, tokenizer

In [78]:
tensor, tokenizer = tokenize(corpus)

[[   2   32   23 ...  720   20   20]
 [   2    5  342 ...    0    0    0]
 [   2    5   32 ...    0    0    0]
 ...
 [   2    9 1890 ...    9 1801   35]
 [   2    9  365 ...    0    0    0]
 [   2    9 1066 ...    9 2337   82]] <keras_preprocessing.text.Tokenizer object at 0x7fb163273f10>


In [79]:
print(tensor[:3, :15])

[[  2  32  23 447 113   6 180   5 107  46   5  57 720  20  20]
 [  2   5 342  15 243  10 154   7  17   3   0   0   0   0   0]
 [  2   5  32  15 312  40  57 107   3   0   0   0   0   0   0]]


In [81]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [82]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2  32  23 447 113   6 180   5 107  46   5  57 720  20]
[ 32  23 447 113   6 180   5 107  46   5  57 720  20  20]


In [83]:
import sklearn.model_selection as sk
enc_train, enc_val, dec_train, dec_val = sk.train_test_split(src_input,tgt_input,test_size=0.2, random_state = 42)

In [84]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140788, 14)
Target Train: (140788, 14)


## 모델 만들기
- 모델의 Embedding Size와 Hidden Size를 조절하며 
- 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)
- 그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [86]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [87]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [88]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-6.9860609e-05,  2.2318140e-04, -1.8489681e-04, ...,
         -1.1546287e-04,  1.1870121e-04, -3.3311164e-05],
        [ 2.2490125e-04,  2.9553103e-04, -4.3457086e-04, ...,
         -7.2490577e-05,  3.2322010e-04, -1.5864021e-04],
        [ 3.9054060e-04,  3.0914971e-04, -3.6673504e-04, ...,
         -2.7319239e-04,  3.9332377e-04, -1.3237158e-04],
        ...,
        [ 7.2450720e-04,  9.3772321e-04, -1.0935551e-03, ...,
         -2.6842888e-04,  1.8879475e-05,  2.3285867e-04],
        [ 6.9730944e-04,  8.4097649e-04, -1.6356441e-03, ...,
         -1.9161675e-04,  1.8958784e-04,  1.8645922e-04],
        [ 6.8404269e-04,  7.1035587e-04, -2.1389131e-03, ...,
         -1.3348088e-04,  3.2640531e-04,  1.2282719e-04]],

       [[-6.9860609e-05,  2.2318140e-04, -1.8489681e-04, ...,
         -1.1546287e-04,  1.1870121e-04, -3.3311164e-05],
        [-2.6807238e-04,  5.9231392e-05, -9.4585834e-05, ...,
         -7.5797245e-05, 

In [89]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [97]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
history = model.fit(dataset, epochs=20, validation_data=(enc_val, dec_val))

Epoch 1/20
549/549 [==============================] - 78s 142ms/step - loss: 2.9315 - val_loss: 2.9550
Epoch 2/20
549/549 [==============================] - 78s 141ms/step - loss: 2.7356 - val_loss: 2.8652
Epoch 3/20
549/549 [==============================] - 77s 141ms/step - loss: 2.5746 - val_loss: 2.7982
Epoch 4/20
549/549 [==============================] - 78s 141ms/step - loss: 2.4362 - val_loss: 2.7515
Epoch 5/20
549/549 [==============================] - 77s 141ms/step - loss: 2.3145 - val_loss: 2.7123
Epoch 6/20
549/549 [==============================] - 77s 141ms/step - loss: 2.2058 - val_loss: 2.6866
Epoch 7/20
549/549 [==============================] - 77s 141ms/step - loss: 2.1063 - val_loss: 2.6667
Epoch 8/20
549/549 [==============================] - 76s 138ms/step - loss: 2.0142 - val_loss: 2.6496
Epoch 9/20
549/549 [==============================] - 77s 140ms/step - loss: 1.9283 - val_loss: 2.6406
Epoch 10/20
549/549 [==============================] - 78s 141ms/step - l

In [91]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [100]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '